# API Response capture

In [1]:
import requests
import json
import pandas as pd
import geopandas as gpd
import numpy as np
import random
from shapely.geometry import Polygon, Point
#import matplotlib.pyplot as plt
#import contextily as cx

/home/lohitd@nextbillion.ai/anaconda3/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


# Input_values

In [2]:
input_boundary_filename = 'Singapore.geojson'
coordinates_range = 100
nb_key = 'plaintesting'
tomtom_key = 'T4NeSzdsHG07NmEA7AvH2Jv2IL1s3qUp'
mapbox_accesstoken = 'pk.eyJ1Ijoia2FpLWxpdSIsImEiOiJjazduNmtxcTkwcHlyM2RxNjVqdzBqcmdjIn0.ElP1f3BdnelgeXuRR4Q3BA'

# Execution

In [3]:
boundary = gpd.read_file(input_boundary_filename)

In [4]:
boundary_bounds = boundary.bounds
minx = boundary_bounds.loc[0,'minx']
miny = boundary_bounds.loc[0,'miny']
maxx = boundary_bounds.loc[0,'maxx']
maxy = boundary_bounds.loc[0,'maxy']

In [5]:
coordinates = []

for each in range(coordinates_range):
    lng = random.uniform(minx, maxx)
    lat = random.uniform(miny, maxy)
    coordinates.append(f"{lat},{lng}")

In [6]:
origin = coordinates[0:int(len(coordinates)/2)]
destination = coordinates[int(len(coordinates)/2):int(len(coordinates)+1)]

In [7]:
api_input = pd.DataFrame(origin, columns=['origin'])
api_input['destination'] = destination

In [ ]:
len(api_input)

# lat/lng split

In [8]:
# Create two lists for the loop results to be placed
origin_lat = []
origin_lon = []
destination_lat = []
destination_lon = []

# For each row in a varible,
for index, row in api_input.iterrows():
    origin_lat.append(row['origin'].split(',')[0])
    origin_lon.append(row['origin'].split(',')[1])
    destination_lat.append(row['destination'].split(',')[0])
    destination_lon.append(row['destination'].split(',')[1])
        
api_input['origin_lat'] = origin_lat
api_input['origin_lon'] = origin_lon
api_input['destination_lat'] = destination_lat
api_input['destination_lon'] = destination_lon

# nb_request_links

In [9]:
nb_request = []

for index, row in api_input.iterrows():
    nb_request.append(f"https://api.nextbillion.io/directions/json?origin={row['origin_lat']},{row['origin_lon']}&destination={row['destination_lat']},{row['destination_lon']}&key={nb_key}")
    
api_input['nb_request_links'] = nb_request

# nb_responses

In [10]:
nb_request_links = api_input['nb_request_links']
nb_response_text = []

for link in nb_request_links:
    nb_response = requests.get(link)
    nb_response_text.append(nb_response.text)
    
api_input['nb_response'] = pd.Series(nb_response_text)

# tomtom_request_links

In [11]:
tomtom_request = []

for index, row in api_input.iterrows():
    tomtom_request.append(f"https://api.tomtom.com/routing/1/calculateRoute/{row['origin_lat']},{row['origin_lon']}:{row['destination_lat']},{row['destination_lon']}/json?key={tomtom_key}&travelMode=car")
    
api_input['tomtom_request_links'] = tomtom_request

# tomtom_responses

In [12]:
tomtom_request_links = api_input['tomtom_request_links']
tomtom_response_text = []

for link in tomtom_request_links:
    tomtom_response = requests.get(link)
    tomtom_response_text.append(tomtom_response.text)
    
api_input['tomtom_response'] = pd.Series(tomtom_response_text)

# mapbox_request_links

In [13]:
mapbox_request = []

for index, row in api_input.iterrows():
    mapbox_request.append(f"https://api.mapbox.com/directions/v5/mapbox/driving/{row['origin_lon']},{row['origin_lat']};{row['destination_lon']},{row['destination_lat']}?access_token={mapbox_accesstoken}&geometries=geojson&alternatives=false")
    
api_input['mapbox_request_links'] = mapbox_request

# mapbox_responses_links

In [14]:
mapbox_request_links = api_input['mapbox_request_links']

mapbox_response_text = []

for link in mapbox_request_links:
    mapbox_response = requests.get(link)
    mapbox_response_text.append(mapbox_response.text)
    
api_input['mapbox_response'] = pd.Series(mapbox_response_text)

# google_request_links

In [ ]:
#nb_request = []

#for index, row in df.iterrows():
    #nb_request.append(f"https://api.nextbillion.io/directions/json?origin={row['origin_lat']},{row['origin_lon']}&destination={row['destination_lat']},{row['destination_lon']}&key=45bea3792b6e46c1a705f3a8bc3eb33a")
    
#api_input['nb_request_links'] = nb_request

# google_responses_links

In [ ]:
#nb_request_links = api_input['nb_request_links']

#nb_response_dict_list = []

#for link in nb_request_links:
    #nb_response = requests.get(link)
    #nb_response_dict_list.append(dict(json.loads(nb_response.text)))
    # PC_Miler_requests

# PC_Miler_requests

In [28]:
pcmiler_requests = []

for index, row in api_input.iterrows():
    pcmiler_requests.append(f"https://pcmiler.alk.com/APIs/REST/v1.0/service.svc/route/routePath?stops={row['origin_lon']}%2C{row['origin_lat']}%3B{row['destination_lon']}%2C{row['destination_lat']}&vehType=0&routeType=0&hwyOnly=false&overrideClass=0&distUnits=0&avoidTolls=false&openBorders=true&overrideRestrict=false&hazMat=0&hubRouting=false&vehDimUnits=0&vehHeight=13%276%22&vehLength=48%270%22&vehWidth=96%22&vehWeight=80000&axles=5&LCV=false&region=4&dataset=PCM_NA&useTraffic=false&ferryDiscourage=false&afSetIDs=&sideOfStreetAdherence=0&useSites=false")
    
api_input['pcmiler_requests'] = pcmiler_requests

# PC_Miler_responses

In [ ]:
nb_request_links = api_input['nb_request_links']

nb_response_dict_list = []

for link in nb_request_links:
    nb_response = requests.get(link)
    nb_response_dict_list.append(dict(json.loads(nb_response.text)))
    time.sleep(3)
    
api_input['nb_response_dict'] = pd.Series(nb_response_dict_list)

# Drop api responses with errors

In [15]:
api_input = api_input[api_input['nb_response'] != '{"msg":"auth error","status":"401"}']

In [16]:
api_input = api_input[api_input['tomtom_response'] !='{"formatVersion":"0.0.12","error":{"description":"Engine error while executing route request: NO_ROUTE_FOUND"},"detailedError":{"message":"Engine error while executing route request: NO_ROUTE_FOUND","code":"NO_ROUTE_FOUND"}}']

In [17]:
api_input = api_input[api_input['mapbox_response'] !='{"code":"NoRoute","message":"No route found","routes":[]}']

# Distance&Duration

# nb_distance&duration

In [18]:
nb_distance = []
nb_duration = []

for index, row in api_input.iterrows():
    nb_response = json.loads(row['nb_response'])
    nb_distance.append(nb_response['routes'][0]['distance'])
    nb_duration.append(nb_response['routes'][0]['duration'])
    
api_input['nb_distance'] = pd.Series(nb_distance)
api_input['nb_duration'] = pd.Series(nb_duration)

# tomtom_distance&duration

In [19]:
tomtom_distance = []
tomtom_duration = []

for index, row in api_input.iterrows():
    tomtom_response = json.loads(row['tomtom_response'])
    tomtom_distance.append(tomtom_response['routes'][0]['summary']['lengthInMeters'])
    tomtom_duration.append(tomtom_response['routes'][0]['summary']['travelTimeInSeconds'])
    
api_input['tomtom_distance'] = pd.Series(tomtom_distance)
api_input['tomtom_duration'] = pd.Series(tomtom_duration)

# mapbox_distance&duration

In [20]:
mapbox_distance = []
mapbox_duration = []

for index, row in api_input.iterrows():
    mapbox_response = json.loads(row['mapbox_response'])
    mapbox_distance.append(mapbox_response['routes'][0]['distance'])
    mapbox_duration.append(mapbox_response['routes'][0]['duration'])
    
api_input['mapbox_distance'] = pd.Series(mapbox_distance)
api_input['mapbox_duration'] = pd.Series(mapbox_duration)

In [21]:
api_input.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26 entries, 0 to 49
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   origin                26 non-null     object 
 1   destination           26 non-null     object 
 2   origin_lat            26 non-null     object 
 3   origin_lon            26 non-null     object 
 4   destination_lat       26 non-null     object 
 5   destination_lon       26 non-null     object 
 6   nb_request_links      26 non-null     object 
 7   nb_response           26 non-null     object 
 8   tomtom_request_links  26 non-null     object 
 9   tomtom_response       26 non-null     object 
 10  mapbox_request_links  26 non-null     object 
 11  mapbox_response       26 non-null     object 
 12  nb_distance           13 non-null     float64
 13  nb_duration           13 non-null     float64
 14  tomtom_distance       13 non-null     float64
 15  tomtom_duration       13 

# PC_Miler_distance_duration

In [ ]:
nb_distance = []
nb_duration = []

for index, row in api_input.iterrows():
    nb_response = dict(row['nb_response_dict'])
    nb_response_routes = nb_response['routes']
    nb_distance.append(nb_response_routes[0]['distance'])
    nb_duration.append(nb_response_routes[0]['duration'])
    
api_input['nb_distance'] = pd.Series(nb_distance)
api_input['nb_duration'] = pd.Series(nb_duration)

# Output

In [22]:
output = api_input[['origin','destination','nb_request_links','nb_response','tomtom_request_links','tomtom_response','mapbox_request_links','mapbox_response','nb_distance','nb_duration','tomtom_distance','tomtom_duration','mapbox_distance','mapbox_duration']]

In [24]:
output.to_csv('output.csv', index=False)

# google_distance&duration

# google_startaddress&endaddress